In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set(rc={'figure.figsize':(11.7,8.27)})
import os

In [2]:
user_data = pd.read_csv("enth_new_cols_z.csv")

In [3]:
sensor_cols = list(user_data.columns)[26:73]
sensor_cols

['Ubibot Temperature(℃)',
 'Ubibot Humidity(%)',
 'Ubibot Light(lux)',
 'Ubibot Voltage(V)',
 'Ubibot WIFI RSSI(dBm)',
 'Ubibot GSM RSSI(dBm)',
 'Ubibot EXT1 Temperature(℃)',
 'Ubibot EXT2 Temperature(℃)',
 'Ubibot RS485 Temperature(℃)',
 'Ubibot RS485 Humidity(%)',
 'BMS VAV2-3|Temp at space_id: 201',
 'BMS VAV2-4|Temp at space_id: 201',
 'BMS VAV2-5|Temp at space_id: 201',
 'BMS VAV2-6|Temp at space_id: 201',
 'BMS VAV2-18|Temp at space_id: 202',
 'BMS VAV2-20|Temp at space_id: 203',
 'BMS VAV2-7N8|Temp at space_id: 204',
 'BMS VAV2-9N27|Temp at space_id: 204',
 'BMS VAV2-22|Temp at space_id: 206',
 'BMS VAV2-17|Temp at space_id: 207',
 'BMS VAV2-16|Temp at space_id: 207',
 'BMS VAV2-12|Temp at space_id: 208',
 'Awair Score',
 'Awair Temp',
 'Awair Humid',
 'Awair CO2',
 'Awair VOC',
 'Awair PM25',
 'Awair Light',
 'Awair Noise',
 'Awair Device UUID',
 'Sensing Humidity',
 'Sensing Light',
 'Sensing Noise',
 'Sensing Temperature',
 'PurpleAir 0.3um count',
 'PurpleAir 0.5um count',
 

In [4]:
temps = list(filter(lambda x: "temp" in x.lower(), sensor_cols))
temps

['Ubibot Temperature(℃)',
 'Ubibot EXT1 Temperature(℃)',
 'Ubibot EXT2 Temperature(℃)',
 'Ubibot RS485 Temperature(℃)',
 'BMS VAV2-3|Temp at space_id: 201',
 'BMS VAV2-4|Temp at space_id: 201',
 'BMS VAV2-5|Temp at space_id: 201',
 'BMS VAV2-6|Temp at space_id: 201',
 'BMS VAV2-18|Temp at space_id: 202',
 'BMS VAV2-20|Temp at space_id: 203',
 'BMS VAV2-7N8|Temp at space_id: 204',
 'BMS VAV2-9N27|Temp at space_id: 204',
 'BMS VAV2-22|Temp at space_id: 206',
 'BMS VAV2-17|Temp at space_id: 207',
 'BMS VAV2-16|Temp at space_id: 207',
 'BMS VAV2-12|Temp at space_id: 208',
 'Awair Temp',
 'Sensing Temperature',
 'PurpleAir Temperature']

In [5]:
humidities = list(filter(lambda x: "humid" in x.lower(), sensor_cols))
humidities

['Ubibot Humidity(%)',
 'Ubibot RS485 Humidity(%)',
 'Awair Humid',
 'Sensing Humidity',
 'PurpleAir Humidity']

In [6]:
def app_sensor_vals():
    """
    find the appropriate Zone temp, light, and humidity
    prioritising data from ubibot
    """
    user_data["Zone Temp"] = np.nan
    user_data["Zone Humidity"] = np.nan
    for row in range(len(user_data)): #fill in columns for areas with sensors first
        if user_data.at[row, "Space ID"] >= 0:
            for temp in temps[:-1]: #exclude purpleair
                if type(user_data.at[row, temp]) is not str and not np.isnan(user_data.at[row, temp]):
                    user_data.at[row, "Zone Temp"] = user_data.at[row, temp]
                    if "Ubibot" in temp:
                        break 
            for humid in humidities[:-1]: #exclude purpleair
                if type(user_data.at[row, humid]) is not str and not np.isnan(user_data.at[row, humid]):
                    user_data.at[row, "Zone Humidity"] = user_data.at[row, humid]
                    if "Ubibot" in humid:
                        break
    purpleair_temp = temps[-1]
    purpleair_hum = humidities[-1]
    for row in range(len(user_data)): #fill in columns for OUTDOOR areas without sensors using purpleair
        if user_data.at[row, "Space ID"] >= 0:
            if user_data.at[row, "Fitbit Indoor/outdoor"] == 9:
                if np.isnan(user_data.at[row, "Zone Temp"]):
                            user_data.at[row, "Zone Temp"] = user_data.at[row, purpleair_temp]
                if np.isnan(user_data.at[row, "Zone Humidity"]):
                            user_data.at[row, "Zone Humidity"] = user_data.at[row, purpleair_hum]
    return user_data

In [7]:
user_data = app_sensor_vals()

In [8]:
user_data

,time,Space ID,Faculty Name,Fitbit Air Velocity,Fitbit Body Presence,Fitbit Change,Fitbit Clothing,Fitbit Comfort,Fitbit Heart Rate,Fitbit Indoor/outdoor,...,"Dependable, self-disciplined","Anxious, easily upset","Open to new experience, complex","Reserved, quiet","Sympathetic, warm","Disorganized, careless","Calm, emotionally stable","Conventional, unreactive",Zone Temp,Zone Humidity
0,2021-03-17 15:30:00+08:00,2,SDE4,10.0,True,10.0,10.0,10.0,87.0,11.0,...,6,3,5,5,5,5,5,2,NaN,NaN
1,2021-03-17 15:33:00+08:00,2,SDE4,10.0,True,10.0,10.0,10.0,67.0,11.0,...,6,3,5,5,5,5,5,2,NaN,NaN
2,2021-03-17 15:53:00+08:00,2,SDE4,10.0,True,10.0,10.0,9.0,87.0,9.0,...,6,3,5,5,5,5,5,2,NaN,NaN
3,2021-03-17 16:08:00+08:00,210,SDE2,11.0,True,10.0,10.0,10.0,70.0,11.0,...,6,3,5,5,5,5,5,2,NaN,NaN
4,2021-03-17 16:23:00+08:00,210,SDE2,11.0,True,10.0,10.0,10.0,66.0,11.0,...,6,3,5,5,5,5,5,2,24.919891,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1398,2021-04-30 17:44:00+08:00,-1,NaN,10.0,True,10.0,9.0,10.0,74.0,11.0,...,3,1,7,1,7,3,5,2,NaN,NaN
1399,2021-04-30 18:01:00+08:00,103,SDE1,10.0,True,10.0,9.0,9.0,63.0,11.0,...,3,1,7,1,7,3,5,2,NaN,NaN
1400,2021-04-30 18:17:00+08:00,103,SDE1,10.0,True,10.0,9.0,10.0,74.0,11.0,...,3,1,7,1,7,3,5,2,NaN,NaN
1401,2021-04-30 18:19:00+08:00,-1,NaN,10.0,True,10.0,9.0,10.0,67.0,11.0,...,3,1,7,1,7,3,5,2,NaN,NaN


In [9]:
survey_dir = f"{os.getcwd()}/onboarding"
survey_file = os.listdir(survey_dir)[0]
survey_results = pd.read_csv(f"{survey_dir}/{survey_file}")\
.rename(columns={"Experiment tag (for example enth99)" : "user_id"})\
.set_index("user_id")

In [10]:
survey_results
user_data = user_data.drop(columns=survey_results.columns[5:])
#to include survey results, comment the above line and restart Main.ipynb and this notebook

In [11]:
user_data = user_data.set_index("time")

In [12]:
user_data

,Space ID,Faculty Name,Fitbit Air Velocity,Fitbit Body Presence,Fitbit Change,Fitbit Clothing,Fitbit Comfort,Fitbit Heart Rate,Fitbit Indoor/outdoor,Fitbit Latitude,...,PurpleAir Pressure,PurpleAir Temperature,User ID,Year of birth (for example 1995),Sex,Height (cm),Weight (kg),Shoulder width (cm),Zone Temp,Zone Humidity
time,,,,,,,,,,,,,,,,,,,,,
2021-03-17 15:30:00+08:00,2,SDE4,10.0,True,10.0,10.0,10.0,87.0,11.0,NaN,...,NaN,NaN,enth15,1998,Male,172,64.2,107,NaN,NaN
2021-03-17 15:33:00+08:00,2,SDE4,10.0,True,10.0,10.0,10.0,67.0,11.0,NaN,...,NaN,NaN,enth15,1998,Male,172,64.2,107,NaN,NaN
2021-03-17 15:53:00+08:00,2,SDE4,10.0,True,10.0,10.0,9.0,87.0,9.0,NaN,...,NaN,NaN,enth15,1998,Male,172,64.2,107,NaN,NaN
2021-03-17 16:08:00+08:00,210,SDE2,11.0,True,10.0,10.0,10.0,70.0,11.0,1.297642,...,1003.94,28.89,enth15,1998,Male,172,64.2,107,NaN,NaN
2021-03-17 16:23:00+08:00,210,SDE2,11.0,True,10.0,10.0,10.0,66.0,11.0,1.297630,...,NaN,NaN,enth15,1998,Male,172,64.2,107,24.919891,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-30 17:44:00+08:00,-1,NaN,10.0,True,10.0,9.0,10.0,74.0,11.0,1.297467,...,NaN,NaN,enth02,1999,Male,180,94.0,116,NaN,NaN
2021-04-30 18:01:00+08:00,103,SDE1,10.0,True,10.0,9.0,9.0,63.0,11.0,1.297465,...,NaN,NaN,enth02,1999,Male,180,94.0,116,NaN,NaN
2021-04-30 18:17:00+08:00,103,SDE1,10.0,True,10.0,9.0,10.0,74.0,11.0,1.297467,...,1006.35,27.22,enth02,1999,Male,180,94.0,116,NaN,NaN


In [13]:
user_data.to_csv("enth_new_cols_z.csv")

# Renaming columns and dropping some debugging columns

In [14]:
user_data_cleaned = user_data.copy()
drop_cols = [
    'Fitbit Latitude',
    'Fitbit Longitude',
    'Fitbit Vote Log',
    'Fitbit API Heart Rate',
    'Fitbit API Steps',
    'Steerpath Accuracy',
    'Steerpath Space',
    'Sensing Humidity',
    'Sensing Light',
    'Sensing Noise',
    'Sensing Temperature',
    'Ubibot Voltage(V)',
    'Ubibot WIFI RSSI(dBm)',
    'Ubibot GSM RSSI(dBm)',
    'Ubibot EXT1 Temperature(℃)',
    'Ubibot EXT2 Temperature(℃)',
    'Ubibot RS485 Temperature(℃)',
    'Ubibot RS485 Humidity(%)',
    'Ubibot Temperature(℃)', 
    'Ubibot Humidity(%)',
    'Ubibot Light(lux)',
    'BMS VAV2-3|Temp at space_id: 201',
    'BMS VAV2-4|Temp at space_id: 201',
    'BMS VAV2-5|Temp at space_id: 201',
    'BMS VAV2-6|Temp at space_id: 201',
    'BMS VAV2-18|Temp at space_id: 202',
    'BMS VAV2-20|Temp at space_id: 203',
    'BMS VAV2-7N8|Temp at space_id: 204',
    'BMS VAV2-9N27|Temp at space_id: 204',
    'BMS VAV2-22|Temp at space_id: 206',
    'BMS VAV2-17|Temp at space_id: 207',
    'BMS VAV2-16|Temp at space_id: 207',
    'BMS VAV2-12|Temp at space_id: 208',
    'Awair Score',
    'Awair Temp',
    'Awair Humid',
    'Awair Light',
    'Awair Device UUID',
    'Year of birth (for example 1995)', 
    'Sex', 
    'Height (cm)', 
    'Weight (kg)',
    'Shoulder circumference (cm)',
]
user_data_cleaned = user_data_cleaned.drop(drop_cols, axis=1)
print(user_data_cleaned.columns)

Index(['Space ID', 'Faculty Name', 'Fitbit Air Velocity',
       'Fitbit Body Presence', 'Fitbit Change', 'Fitbit Clothing',
       'Fitbit Comfort', 'Fitbit Heart Rate', 'Fitbit Indoor/outdoor',
       'Fitbit Metabolism', 'Fitbit Response Speed',
       'Fitbit Resting Heart Rate', 'Fitbit Thermal',
       'iButton Ambient Temp(℃)', 'Steerpath Floor', 'Steerpath Latitude',
       'Steerpath Longitude', 'iButton Skin Temp(℃)', 'Awair CO2', 'Awair VOC',
       'Awair PM25', 'Awair Noise', 'PurpleAir 0.3um count',
       'PurpleAir 0.5um count', 'PurpleAir 1.0um count',
       'PurpleAir 10.0um count', 'PurpleAir 2.5um count',
       'PurpleAir 5.0um count', 'PurpleAir Humidity', 'PurpleAir PM1.0',
       'PurpleAir PM10.0', 'PurpleAir PM2.5', 'PurpleAir Pressure',
       'PurpleAir Temperature', 'User ID', 'Zone Temp', 'Zone Humidity'],
      dtype='object')


In [15]:
new_cols = [
    'space_id', 
    'building_name', 
    'air_vel',
    'body_presence',
    'change', 
    'clothing',
    'comfort',
    'heartrate',
    'indoor/outdoor', 
    'met', 
    'response_speed',
    'resting_heartrate',
    'thermal', 
    'nb_temp',
    'indoor_floor',
    'indoor_latitude', 
    'indoor_longitude',
    'skin_temp',
    'co2_indoor', 
    'voc_indoor', 
    'pm25_indoor', 
    'noise_indoor',
    '0.3um_count_outdoor', 
    '0.5um_count_outdoor',
    '1.0um_count_outdoor', 
    '10.0um_count_outdoor',
    '2.5um_count_outdoor', 
    '5.0um_count_outdoor', 
    'humidity_outdoor',
    'pm1.0_outdoor',
    'pm10.0_outdoor', 
    'pm2.5_outdoor', 
    'pressure_outdoor', 
    'temp_outdoor',
    'user_id',
    'temp_zone',
    'humidity_zone',
]
user_data_cleaned.columns = new_cols
user_data_cleaned.to_csv("enth_tabular_merged.csv")
!cp enth_tabular_merged.csv ../processed/